In [1]:
import numpy as np
import json
import sys
import pickle
import time
import cvxopt as cvx
# from cvxopt import matrix, solvers

In [2]:
def read_data (datafile="../dataset/SVM/train.csv", entryno=1):
    lines = np.array([(line.rstrip('\n')).split(',') for line in open(datafile)], dtype=np.int64)
    lines = lines[ (lines[:,-1] == entryno) | (lines[:,-1] == entryno+1) ]
    np.random.seed(0) # Deterministically random
    np.random.shuffle(lines)
#     print (lines[:10])
#     lines = lines[:200,:]
    X = lines[:,:-1] / 255.0
    Y = lines[:,-1] * 2 - 3 # for having y = {-1, 1}
    
#     print (X.shape, X)
#     np.set_printoptions(threshold=np.nan)
#     print (Y.shape, Y)
#     print (lines.shape, lines)
#     print (np.max(X[0]), np.min(X[0]))

    return X, Y
    
X, Y = read_data('../dataset/SVM/train.csv')
testX, testY = read_data('../dataset/SVM/test.csv')
print (X.shape, Y.shape, testX.shape, testY.shape)

(4000, 784) (4000,) (2167, 784) (2167,)


In [3]:
# LibSVM
from libsvm.python.svmutil import *

# prob = svm_problem(Y, X)

# param = svm_parameter()
# param.kernel_type = LINEAR
# param.C = 1

# m=svm_train(prob, param)

# correct = 0
# svm_predict(Y, X, m)[0]
# result = testY == svm_predict(testY, testX, m)[0]
# np.sum(result)

In [7]:
# Finding the SVM alphas
X = np.matrix(X)
Y = np.matrix(Y).T

m, n = X.shape
print (X.shape, Y.shape, m, n)

(4000, 784) (1, 4000) 4000 784


In [13]:
X[0] * X[1].T
M = np.zeros((m, m))
for i in range (m):
    for j in range (m):
        M[i][j] = float (X[i] * X[j].T)
        
print (M)

[[ 76.74978854  46.94712803  17.29138024 ...  21.15249519  30.8811534
   29.73359477]
 [ 46.94712803  61.93600923  22.13290273 ...  28.19813918  32.19226451
   28.4963168 ]
 [ 17.29138024  22.13290273  49.81311803 ...  50.78248366  22.85188774
   31.20730488]
 ...
 [ 21.15249519  28.19813918  50.78248366 ... 101.24009227  41.31810842
   59.17339485]
 [ 30.8811534   32.19226451  22.85188774 ...  41.31810842  72.36329104
   44.5817301 ]
 [ 29.73359477  28.4963168   31.20730488 ...  59.17339485  44.5817301
   86.90568243]]


In [15]:
mQ = np.zeros((m, m))
print (Y[0])
# for i in range (0, m):
#     for j in range (0, m):
#         Q[i][j] = 

[[-1 -1 -1 ...  1  1  1]]


In [5]:
Q = X * X.T * Y * Y.T

p = np.matrix(-np.ones((m, 1)))

G = np.matrix(np.vstack( (-np.identity(m), np.identity(m)) ))
h = np.matrix(np.vstack( (np.zeros((m,1)), np.ones((m,1))) ))

A = Y.T
b = 0

print (Q.shape, p.shape, G.shape, h.shape, A.shape, b)
# print (Q, p, G, h, A, b)

(4000, 4000) (4000, 1) (8000, 4000) (8000, 1) (1, 4000) 0


In [6]:
Q = cvx.matrix(Q)
p = cvx.matrix(p)
G = cvx.matrix(G)
h = cvx.matrix(h)
A = cvx.matrix(A, (1, m), 'd')
print (A, np.array(A).shape)
b = cvx.matrix(b, (1,1), 'd')

sol = cvx.solvers.qp(P=Q, q=p, G=G, h=h, A=A, b=b)

[-1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00  1.00e+00 ... ]
 (1, 4000)


ValueError: Rank(A) < p or Rank([P; A; G]) < n